# Psoriasis Disease Detection using Transfer Learning Models

Psoriasis, affecting 125 million globally, causes rashes, scaly patches, and itchy skin. Our research creates a dataset with seven psoriasis classes, using InceptionResNetV2 and InceptionV3 for accurate classification.

## Detection using the InceptionResNetV2

Inception-ResNet V2 is a deep convolutional neural network combining the strengths of Inception modules and residual connections. It incorporates residual connections to improve training speed and stability, while the Inception modules enhance feature extraction efficiency. This architecture achieves high accuracy in image classification tasks and is widely used in medical image analysis and other applications.

### Importing the Required Libraries and Transfer Learning Models

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import RMSprop

### Importing the Dataset

The dataset includes seven classes of psoriasis diseases sourced from publicly available datasets such as SKIN LESION, ISIC, and DEMANET. It sets paths for training, testing, and validation datasets and initializes data generators. The training generator applies rescaling, shearing, zooming, and horizontal flipping for data augmentation, while the testing and validation generators only rescale the images. All generators resize images to 224x224 pixels, prepare them in batches of 32, and use categorical classification mode. The output shows the dataset statistics: 2800 images for training, 608 for validation, and 597 for testing, distributed among the seven psoriasis classes.

In [ ]:
train_path = "\path"
test_path  = "\path"
val_path   = "\path"

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

val_generator = test_datagen.flow_from_directory(val_path,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 2800 images belonging to 7 classes.
Found 608 images belonging to 7 classes.
Found 597 images belonging to 7 classes.


### Building the InceptionResNetV2 Model

Initializing a transfer learning model using Inception-ResNet V2 pre-trained on ImageNet for image classification. It excludes the top classification layers initially and adds a global average pooling layer to reduce spatial dimensions, followed by a dense layer with 1024 units and ReLU activation. The final dense layer has softmax activation for categorical prediction based on the number of classes in the training data. The model architecture is defined with the base model's input and custom output layers. All layers from the Inception-ResNet V2 base model are frozen to retain pre-learned features during training on new data.

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

### Compiling the Model with RMSprop Optimizer

In [ ]:
model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the Model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

C:\Users\DSAI\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
87/87 [==============================] - 36s 275ms/step - loss: 2.1078 - accuracy: 0.5560 - val_loss: 0.8400 - val_accuracy: 0.6632
Epoch 2/20
87/87 [==============================] - 16s 180ms/step - loss: 0.8306 - accuracy: 0.7265 - val_loss: 0.5047 - val_accuracy: 0.8247
Epoch 3/20
87/87 [==============================] - 16s 183ms/step - loss: 0.6425 - accuracy: 0.7735 - val_loss: 0.5191 - val_accuracy: 0.8160
Epoch 4/20
87/87 [==============================] - 16s 181ms/step - loss: 0.5265 - accuracy: 0.8121 - val_loss: 0.3319 - val_accuracy: 0.8663
Epoch 5/20
87/87 [==============================] - 16s 181ms/step - loss: 0.4246 - accuracy: 0.8504 - val_loss: 0.3255 - val_accuracy: 0.8837
Epoch 6/20
87/87 [==============================] - 16s 180ms/step - loss: 0.3778 - accuracy: 0.8645 - val_loss: 0.2781 - val_accuracy: 0.8958
Epoch 7/20
87/87 [==============================] - 16s 182ms/step - loss: 0.3256 - accuracy: 0.8913 - val_loss: 0.4347 - val_accuracy: 0.8455

### Building and Compiling the same Model using the Adam Optimizer

Initializing a transfer learning model using Inception-ResNet V2, pretrained on ImageNet, for image classification tasks. It excludes the top classification layers by default, adds a global average pooling layer for spatial reduction, and appends a dense layer with 1024 units and ReLU activation. The final dense layer, using softmax activation, predicts categories based on the number of classes in the training data. The model is compiled with the Adam optimizer, learning rate set to 0.001, categorical cross-entropy loss function, and accuracy as the evaluation metric. All layers from the Inception-ResNet V2 base model are frozen to retain pre-learned features during training on new data.

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

C:\Users\DSAI\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
259/259 [==============================] - 72s 259ms/step - loss: 2.0898 - accuracy: 0.3736 - val_loss: 1.6732 - val_accuracy: 0.4626
Epoch 2/20
259/259 [==============================] - 66s 253ms/step - loss: 1.5719 - accuracy: 0.4898 - val_loss: 1.4989 - val_accuracy: 0.5098
Epoch 3/20
259/259 [==============================] - 65s 250ms/step - loss: 1.4250 - accuracy: 0.5262 - val_loss: 1.3968 - val_accuracy: 0.5461
Epoch 4/20
259/259 [==============================] - 65s 250ms/step - loss: 1.3235 - accuracy: 0.5535 - val_loss: 1.3995 - val_accuracy: 0.5487
Epoch 5/20
259/259 [==============================] - 65s 251ms/step - loss: 1.2776 - accuracy: 0.5716 - val_loss: 1.3111 - val_accuracy: 0.5705
Epoch 6/20
259/259 [==============================] - 65s 252ms/step - loss: 1.2044 - accuracy: 0.5926 - val_loss: 1.3606 - val_accuracy: 0.5647
Epoch 7/20
259/259 [==============================] - 65s 251ms/step - loss: 1.1800 - accuracy: 0.5932 - val_loss: 1.1676 - val_ac

Evaluating the trained Inception-ResNet V2 model on the training, validation, and test datasets using the evaluate method. It computes and retrieves the loss and accuracy metrics for each dataset. The train_generator, val_generator, and test_generator are used as input to evaluate the model on their respective datasets. The results are printed to the console, displaying the training loss and accuracy, validation loss and accuracy, and test loss and accuracy.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 2s 117ms/step - loss: 0.1693 - accuracy: 0.9490
Train loss:, val loss: test loss:  0.08580939471721649 0.1908438354730606 0.16933879256248474
Train accuracy: val accuracy:Test accuracy: 0.9674999713897705 0.9380234479904175 0.9490131735801697


## Detection using the InceptionV3 Model

InceptionV3 is a deep convolutional neural network architecture developed by Google, renowned for its efficiency and accuracy in image classification and object detection tasks. It utilizes various convolutional filter sizes and auxiliary classifiers to capture multi-scale features effectively, making it suitable for diverse computer vision applications.

### Building the InceptionV3 Model and Compiling the Model using Adam Optimizer

Building a transfer learning model using the InceptionV3 architecture pre-trained on ImageNet for image classification tasks. It initializes InceptionV3 without its top classification layers and adds a global average pooling layer followed by a dense layer with 1024 units and ReLU activation. The final dense layer predicts classes based on the number of classes in the training data using softmax activation. The model's base layers are frozen to retain pre-trained weights. It is compiled with the Adam optimizer, a learning rate of 0.001, categorical cross-entropy loss, and accuracy as the evaluation metric.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/20


C:\Users\DSAI\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


87/87 [==============================] - 17s 174ms/step - loss: 1.0717 - accuracy: 0.6770 - val_loss: 0.5600 - val_accuracy: 0.8142
Epoch 2/20
87/87 [==============================] - 14s 165ms/step - loss: 0.4492 - accuracy: 0.8483 - val_loss: 0.3915 - val_accuracy: 0.8646
Epoch 3/20
87/87 [==============================] - 14s 163ms/step - loss: 0.3631 - accuracy: 0.8714 - val_loss: 0.3801 - val_accuracy: 0.8628
Epoch 4/20
87/87 [==============================] - 14s 166ms/step - loss: 0.2705 - accuracy: 0.9053 - val_loss: 0.3193 - val_accuracy: 0.8941
Epoch 5/20
87/87 [==============================] - 15s 168ms/step - loss: 0.2380 - accuracy: 0.9115 - val_loss: 0.2848 - val_accuracy: 0.9132
Epoch 6/20
87/87 [==============================] - 15s 166ms/step - loss: 0.2083 - accuracy: 0.9267 - val_loss: 0.2901 - val_accuracy: 0.9062
Epoch 7/20
87/87 [==============================] - 14s 165ms/step - loss: 0.2222 - accuracy: 0.9184 - val_loss: 0.2576 - val_accuracy: 0.9115
Epoch 8/20

Evaluating the trained InceptionV3 model on the training, validation, and test datasets using the evaluate method. It computes and retrieves the loss and accuracy metrics for each dataset and prints them to the console.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 63ms/step - loss: 0.1200 - accuracy: 0.9539
Train loss:, val loss: test loss:  0.0499376580119133 0.1915629357099533 0.11997473984956741
Train accuracy: val accuracy:Test accuracy: 0.9839285612106323 0.9430485963821411 0.9539473652839661


### Building the InceptionV3 Model and Compiling the Model using RMSprop Optimizer

Building a transfer learning model using the InceptionV3 architecture pre-trained on ImageNet for image classification tasks. It initializes InceptionV3 without its top classification layers and adds a global average pooling layer followed by a dense layer with 1024 units and ReLU activation. The final dense layer predicts classes based on the number of classes in the training data using softmax activation. The model's base layers are frozen to retain pre-trained weights. It is compiled with the RMSprop optimizer, a learning rate of 0.001, categorical cross-entropy loss, and accuracy as the evaluation metric.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = base_model.layers.DropOut(0.1)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=30,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/30
87/87 [==============================] - 18s 177ms/step - loss: 1.9657 - accuracy: 0.5697 - val_loss: 0.6459 - val_accuracy: 0.7847
Epoch 2/30
87/87 [==============================] - 15s 167ms/step - loss: 0.7905 - accuracy: 0.7341 - val_loss: 0.4434 - val_accuracy: 0.8403
Epoch 3/30
87/87 [==============================] - 15s 166ms/step - loss: 0.5491 - accuracy: 0.8056 - val_loss: 0.3915 - val_accuracy: 0.8611
Epoch 4/30
87/87 [==============================] - 15s 168ms/step - loss: 0.4815 - accuracy: 0.8277 - val_loss: 0.3945 - val_accuracy: 0.8420
Epoch 5/30
87/87 [==============================] - 15s 167ms/step - loss: 0.3519 - accuracy: 0.8725 - val_loss: 0.4996 - val_accuracy: 0.8490
Epoch 6/30
87/87 [==============================] - 14s 165ms/step - loss: 0.3235 - accuracy: 0.8833 - val_loss: 0.3708 - val_accuracy: 0.8785
Epoch 7/30
87/87 [==============================] - 14s 166ms/step - loss: 0.2896 - accuracy: 0.8970 - val_loss: 0.2083 - val_accuracy: 0.9288

Evaluating the trained InceptionV3 model on the training, validation, and test datasets using the evaluate method. It computes and retrieves the loss and accuracy metrics for each dataset and prints them to the console in a clear format. Adjustments have been made to ensure the print statements correctly display the evaluation results.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 63ms/step - loss: 0.1590 - accuracy: 0.9622
Train loss:, val loss: test loss:  0.11254952847957611 0.24523282051086426 0.15901249647140503
Train accuracy: val accuracy:Test accuracy: 0.9710714221000671 0.94807368516922 0.9621710777282715


### Building the InceptionV3 Model using BatchNormalization and Compling the Model using Adam Optimizer

Adding a batch normalization layer and dropout regularization to improve generalization and reduce overfitting to this model architecture. The final dense layer with 512 units and ReLU activation enhances feature extraction capabilities. The base layers of InceptionV3 remain frozen to retain pre-trained weights. The model is compiled with the Adam optimizer, a learning rate of 0.001, categorical cross-entropy loss function, and accuracy metric for evaluation during training.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x=  Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)

predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/20
87/87 [==============================] - 18s 176ms/step - loss: 0.9105 - accuracy: 0.7446 - val_loss: 0.4410 - val_accuracy: 0.8698
Epoch 2/20
87/87 [==============================] - 14s 165ms/step - loss: 0.4645 - accuracy: 0.8537 - val_loss: 0.4014 - val_accuracy: 0.8785
Epoch 3/20
87/87 [==============================] - 14s 165ms/step - loss: 0.3591 - accuracy: 0.8978 - val_loss: 0.2829 - val_accuracy: 0.9097
Epoch 4/20
87/87 [==============================] - 15s 168ms/step - loss: 0.2993 - accuracy: 0.9111 - val_loss: 0.3165 - val_accuracy: 0.9028
Epoch 5/20
87/87 [==============================] - 15s 169ms/step - loss: 0.2252 - accuracy: 0.9332 - val_loss: 0.2499 - val_accuracy: 0.9392
Epoch 6/20
87/87 [==============================] - 15s 166ms/step - loss: 0.2298 - accuracy: 0.9281 - val_loss: 0.2324 - val_accuracy: 0.9497
Epoch 7/20
87/87 [==============================] - 14s 166ms/step - loss: 0.2257 - accuracy: 0.9364 - val_loss: 0.2844 - val_accuracy: 0.9427

Evaluating the trained model (model) on the training (train_generator), validation (val_generator), and test (test_generator) datasets using the evaluate method. It calculates and retrieves the loss and accuracy metrics for each dataset and prints them in a clear format. Adjustments have been made to ensure the print statements display the evaluation results correctly.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 63ms/step - loss: 0.0987 - accuracy: 0.9704
Train loss:, val loss: test loss:  0.03159834071993828 0.19860288500785828 0.09868647158145905
Train accuracy: val accuracy:Test accuracy: 0.9903571605682373 0.9547738432884216 0.9703947305679321


### InceptionV3 Model with 768 Neurons

Incorporating a batch normalization layer and dropout regularization (40%) to enhance training stability and prevent overfitting. The final dense layer with 768 units and ReLU activation aims to capture more complex features from the base model's output. The base layers of InceptionV3 remain frozen to retain pre-trained weights. The model is compiled with the Adam optimizer, a learning rate of 0.001, categorical cross-entropy loss function, and accuracy metric for evaluation during training. Adjustments have been made for clarity and to align with best practices in model building and regularization techniques.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x=  Dropout(0.4)(x)
x = Dense(768, activation='relu')(x)

predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/20
87/87 [==============================] - 17s 175ms/step - loss: 1.0248 - accuracy: 0.7504 - val_loss: 0.6572 - val_accuracy: 0.7865
Epoch 2/20
87/87 [==============================] - 14s 166ms/step - loss: 0.5005 - accuracy: 0.8562 - val_loss: 0.3954 - val_accuracy: 0.8872
Epoch 3/20
87/87 [==============================] - 14s 164ms/step - loss: 0.3954 - accuracy: 0.8851 - val_loss: 0.3144 - val_accuracy: 0.9253
Epoch 4/20
87/87 [==============================] - 14s 165ms/step - loss: 0.3151 - accuracy: 0.9032 - val_loss: 0.2845 - val_accuracy: 0.9427
Epoch 5/20
87/87 [==============================] - 15s 168ms/step - loss: 0.2600 - accuracy: 0.9155 - val_loss: 0.2216 - val_accuracy: 0.9462
Epoch 6/20
87/87 [==============================] - 15s 168ms/step - loss: 0.2197 - accuracy: 0.9335 - val_loss: 0.2218 - val_accuracy: 0.9514
Epoch 7/20
87/87 [==============================] - 15s 167ms/step - loss: 0.2121 - accuracy: 0.9350 - val_loss: 0.3154 - val_accuracy: 0.9340

Evaluating the trained InceptionV3-based model (model) on the training (train_generator), validation (val_generator), and test (test_generator) datasets using the evaluate method. It calculates and retrieves the loss and accuracy metrics for each dataset and prints them in a clear format. Adjustments have been made to ensure the print statements display the evaluation results correctly.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 61ms/step - loss: 0.1101 - accuracy: 0.9803
Train loss:, val loss: test loss:  0.041993625462055206 0.2340845912694931 0.11014299839735031
Train accuracy: val accuracy:Test accuracy: 0.9850000143051147 0.9597989916801453 0.9802631735801697


### InceptionV3 Model with 1024 Neurons

Building a transfer learning model using the InceptionV3 architecture pre-trained on ImageNet for image classification. It initializes InceptionV3 without its top layers and adds a global average pooling layer, batch normalization for stability, a dropout layer with a rate of 0.4 for regularization, and a dense layer with 1024 units and ReLU activation for feature extraction. The final dense layer predicts classes based on the number of classes in the training data using softmax activation. All layers from the InceptionV3 base model are frozen to retain pre-trained weights. The model is compiled with the Adam optimizer, a learning rate of 0.001, categorical cross-entropy loss function, and accuracy metric for evaluation during training.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x=  Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/20
87/87 [==============================] - 18s 180ms/step - loss: 1.0500 - accuracy: 0.7460 - val_loss: 0.5652 - val_accuracy: 0.8073
Epoch 2/20
87/87 [==============================] - 15s 172ms/step - loss: 0.5548 - accuracy: 0.8551 - val_loss: 0.3622 - val_accuracy: 0.8958
Epoch 3/20
87/87 [==============================] - 15s 172ms/step - loss: 0.4243 - accuracy: 0.8790 - val_loss: 0.3421 - val_accuracy: 0.9028
Epoch 4/20
87/87 [==============================] - 15s 170ms/step - loss: 0.3242 - accuracy: 0.9090 - val_loss: 0.2757 - val_accuracy: 0.9358
Epoch 5/20
87/87 [==============================] - 15s 170ms/step - loss: 0.2872 - accuracy: 0.9162 - val_loss: 0.2527 - val_accuracy: 0.9444
Epoch 6/20
87/87 [==============================] - 15s 172ms/step - loss: 0.2396 - accuracy: 0.9310 - val_loss: 0.3001 - val_accuracy: 0.9375
Epoch 7/20
87/87 [==============================] - 15s 172ms/step - loss: 0.2531 - accuracy: 0.9281 - val_loss: 0.1988 - val_accuracy: 0.9514

Evaluating the trained InceptionV3-based model (model) on the training (train_generator), validation (val_generator), and test (test_generator) datasets using the evaluate method. It calculates and retrieves the loss and accuracy metrics for each dataset and prints them in a clear format. Adjustments have been made to ensure the print statements display the evaluation results correctly.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 63ms/step - loss: 0.1548 - accuracy: 0.9589
Train loss:, val loss: test loss:  0.04572135955095291 0.27109161019325256 0.15479077398777008
Train accuracy: val accuracy:Test accuracy: 0.9871428608894348 0.9497487545013428 0.9588815569877625


### Building the InceptionV3 Model with a Dropout Layer and Compiling using Adagrad Optimizer

Constructing a transfer learning model using the InceptionV3 architecture pretrained on ImageNet for image classification tasks. It initializes InceptionV3 without its fully connected layers (include_top=False) and adds a global average pooling layer to reduce spatial dimensions, followed by batch normalization and a dropout layer (40%) for regularization. A dense layer with 1024 units and ReLU activation further processes the features. Another batch normalization layer is applied before the final dense layer, which predicts classes based on the number of categories in train_generator using softmax activation. All layers from the InceptionV3 base model are frozen to retain pretrained weights. The model is compiled with the Adagrad optimizer, configured with a learning rate of 0.001, categorical cross-entropy loss function, and accuracy metric for model evaluation during training.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x=  Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adagrad(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/20


C:\Users\DSAI\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adagrad.py:81: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


87/87 [==============================] - 18s 178ms/step - loss: 1.6358 - accuracy: 0.4274 - val_loss: 1.0626 - val_accuracy: 0.6684
Epoch 2/20
87/87 [==============================] - 15s 174ms/step - loss: 1.0094 - accuracy: 0.6456 - val_loss: 0.7611 - val_accuracy: 0.7622
Epoch 3/20
87/87 [==============================] - 15s 174ms/step - loss: 0.8143 - accuracy: 0.7287 - val_loss: 0.6125 - val_accuracy: 0.8125
Epoch 4/20
87/87 [==============================] - 15s 173ms/step - loss: 0.7231 - accuracy: 0.7424 - val_loss: 0.5347 - val_accuracy: 0.8281
Epoch 5/20
87/87 [==============================] - 15s 172ms/step - loss: 0.6363 - accuracy: 0.7887 - val_loss: 0.4904 - val_accuracy: 0.8455
Epoch 6/20
87/87 [==============================] - 15s 172ms/step - loss: 0.5997 - accuracy: 0.7912 - val_loss: 0.4342 - val_accuracy: 0.8663
Epoch 7/20
87/87 [==============================] - 15s 173ms/step - loss: 0.5621 - accuracy: 0.7999 - val_loss: 0.4018 - val_accuracy: 0.8750
Epoch 8/20

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 64ms/step - loss: 0.2354 - accuracy: 0.9424
Train loss:, val loss: test loss:  0.1829499453306198 0.2460925430059433 0.23535431921482086
Train accuracy: val accuracy:Test accuracy: 0.9467856884002686 0.9246231317520142 0.9424341917037964


### Building the InceptionV3 Model using two Dropout Layers

Building a transfer learning model using InceptionV3, pretrained on ImageNet, for image classification tasks. The base model is loaded without its fully connected layers (include_top=False), and additional layers are added for regularization and feature extraction. It starts with a dropout layer (dropout rate of 40%) to prevent overfitting, followed by global average pooling to reduce spatial dimensions. Batch normalization layers are applied for stabilizing the training process, and another dropout layer (40%) further regularizes the model. A dense layer with 1024 units and ReLU activation processes the extracted features, followed by another batch normalization layer. The final dense layer uses softmax activation to predict classes based on the number of categories in the train_generator. All layers from the InceptionV3 base model are frozen to retain their pretrained weights. The model is compiled with the Adam optimizer, configured with a learning rate of 0.001, categorical cross-entropy loss function, and accuracy metric for model evaluation during training.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = Dropout(0.4)(x)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x=  Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
# Training the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.n // val_generator.batch_size)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/20
87/87 [==============================] - 18s 180ms/step - loss: 1.0431 - accuracy: 0.7392 - val_loss: 0.5410 - val_accuracy: 0.8247
Epoch 2/20
87/87 [==============================] - 15s 171ms/step - loss: 0.5087 - accuracy: 0.8432 - val_loss: 0.3062 - val_accuracy: 0.9132
Epoch 3/20
87/87 [==============================] - 15s 171ms/step - loss: 0.3461 - accuracy: 0.8866 - val_loss: 0.2656 - val_accuracy: 0.9132
Epoch 4/20
87/87 [==============================] - 15s 175ms/step - loss: 0.2794 - accuracy: 0.9025 - val_loss: 0.2626 - val_accuracy: 0.9219
Epoch 5/20
87/87 [==============================] - 15s 174ms/step - loss: 0.2647 - accuracy: 0.9133 - val_loss: 0.2347 - val_accuracy: 0.9271
Epoch 6/20
87/87 [==============================] - 15s 172ms/step - loss: 0.2032 - accuracy: 0.9288 - val_loss: 0.1714 - val_accuracy: 0.9392
Epoch 7/20
87/87 [==============================] - 15s 171ms/step - loss: 0.1875 - accuracy: 0.9346 - val_loss: 0.1690 - val_accuracy: 0.9497

Evaluating method to assess the performance of your InceptionV3-based model (model) on the training (train_generator), validation (val_generator), and test (test_generator) datasets. It computes and retrieves the loss and accuracy metrics for each dataset and prints them in a clear format. Adjustments have been made to ensure the print statements display the evaluation results correctly.

In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss,val_acc = model.evaluate(val_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Train loss:, val loss: test loss: ', train_loss,val_loss, test_loss)
print('Train accuracy: val accuracy:Test accuracy:', train_acc,val_acc,test_acc)

19/19 [==============================] - 1s 63ms/step - loss: 0.0667 - accuracy: 0.9704
Train loss:, val loss: test loss:  0.02722633071243763 0.13767699897289276 0.066710464656353
Train accuracy: val accuracy:Test accuracy: 0.9907143115997314 0.9614740610122681 0.9703947305679321
